# Introduction to the Trelliscope Python Package
Trelliscope provides a simple mechanism to make a collection of visualizations and display them as interactive [small multiples](https://en.wikipedia.org/wiki/Small_multiple).

For more information about Trelliscopes generally, please refer to the [Introduction to Trelliscope](https://trelliscope.org/trelliscope/articles/trelliscope.html) page that shows the R library syntax. This page will highlight the Python version of the library.

## Pre-Generated Images
The basic principle behind the Trelliscope package is that you specify a data frame that contains figures or images. We refer to each plot (row) of a given visualization (column) as a **panel**, and hence will often refer to a visualization column as a collection of panels.

The simplest way to illustrate what is meant by a data frame of visualizations is to start with an example where the images have already been generated.

An example dataset that comes with the package contains images captured by the Mars rover Curiosity.

In [1]:
# import necessary libraries
import os
import pandas as pd
from trelliscope import Trelliscope

In [2]:
# The mars rover dataset is in the trelliscope/examples/external_data folder
mars_file = os.path.join("external_data", "mars_rover.csv")
mars_df = pd.read_csv(mars_file)
print(mars_df.head())

                  camera  sol  earth_date    class  \
0  Mars Hand Lens Imager  565  2014-03-09  horizon   
1  Mars Hand Lens Imager  565  2014-03-09  horizon   
2  Mars Hand Lens Imager  568  2014-03-12    wheel   
3  Mars Hand Lens Imager  568  2014-03-12    wheel   
4  Mars Hand Lens Imager  568  2014-03-12    wheel   

                                             img_src  
0  http://mars.jpl.nasa.gov/msl-raw-images/msss/0...  
1  http://mars.jpl.nasa.gov/msl-raw-images/msss/0...  
2  http://mars.jpl.nasa.gov/msl-raw-images/msss/0...  
3  http://mars.jpl.nasa.gov/msl-raw-images/msss/0...  
4  http://mars.jpl.nasa.gov/msl-raw-images/msss/0...  


## Creating and Viewing a Trelliscope
This data frame has a column that references images on the web, img_src. The other columns contain metadata about these images. We can create a Trelliscope object, write its display information out to the file system, and view it in a browser with the following:

In [3]:
tr = (Trelliscope(mars_df, name="Mars Rover")
        .write_display()
        .view_trelliscope()
)

INFO:root:Using ['camera', 'earth_date', 'class', 'img_src'] to uniquely identify each row of the data.
INFO:root:Saving to /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpyv2xt2z5/mars_rover
DEBUG:root:Inferring Metas: {'camera', 'class', 'img_src', 'earth_date', 'sol'}
DEBUG:root:Successfully inferred metas: ['camera', 'class', 'img_src', 'earth_date', 'sol']
INFO:root:No layout definition supplied. Using Default.
INFO:root:No labels definition supplied. Using Default.
INFO:root:Trelliscope written to `/var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpyv2xt2z5/mars_rover`


## Creating Panel Columns
The previous example shows how to create and view a trelliscope with pre-existing images. The following example shows a more likely use case where we are interesting a creating and viewing plots using Plotly Express.


First, we will import a handful of libraries additional libraries used in this example.

In [4]:
# import necessary libraries
import os
import tempfile
import urllib.request
import zipfile
import pandas as pd
import plotly.express as px

from trelliscope import Trelliscope
from trelliscope.facets import facet_panels
from trelliscope.state import NumberRangeFilterState

In [5]:
# set up some constants for the example
use_small_dataset = True

### Load the Dataset
This example will use a version of the gapminder dataset included with the trelliscope library.

In [6]:
# Load the dataset
gapminder_file = os.path.join("external_data", "gapminder.csv")
gapminder = pd.read_csv(gapminder_file)

if use_small_dataset:
    df = gapminder[:200]
else:
    df = gapminder


### Build panels using `facet_panels`
The `facet_panels` function does a _group by_ and applies a plot function to each of the groups. One way to think about this is that the group by creates a number of small data frames, and then uses the px.scatter function to create a plot for each of these small data frames.

Inspecting the resulting data frame shows that it has been grouped by the appropriate columns and that there is also an image column that is a Plotly Express figure.

In [7]:
# Use facet_panels to create a Plotly Express graphic for each small data frame
panel_df = facet_panels(df, "lifeExp_time", ["country", "continent", "iso_alpha2"], px.scatter, {"x": "year", "y": "lifeExp"})

# Inspecting panel_df shows a dataset grouped by `country`, `continent`, and `iso_alpha2`.
# Notice that there is also a `lifeExp_time` column that is a plotly express figure
panel_df

,,,lifeExp_time
country,continent,iso_alpha2,
Afghanistan,Asia,AF,Figure({\n 'data': [{'hovertemplate': 'year...
Albania,Europe,AL,Figure({\n 'data': [{'hovertemplate': 'year...
Algeria,Africa,DZ,Figure({\n 'data': [{'hovertemplate': 'year...
Angola,Africa,AO,Figure({\n 'data': [{'hovertemplate': 'year...
Argentina,Americas,AR,Figure({\n 'data': [{'hovertemplate': 'year...
Australia,Oceania,AU,Figure({\n 'data': [{'hovertemplate': 'year...
Austria,Europe,AT,Figure({\n 'data': [{'hovertemplate': 'year...
Bahrain,Asia,BH,Figure({\n 'data': [{'hovertemplate': 'year...
Bangladesh,Asia,BD,Figure({\n 'data': [{'hovertemplate': 'year...


In [8]:
# A trelliscope can be prepared and viewed for this data frame
tr = (Trelliscope(panel_df, "Gapminder")
      .write_display()
      .view_trelliscope()
)

INFO:root:Using ['country', 'continent', 'iso_alpha2'] to uniquely identify each row of the data.
INFO:root:Saving to /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpuqlp_9hu/gapminder
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpuqlp_9hu/gapminder/displays/Gapminder/panels/lifeexp_time/afghanistan_asia_af.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpuqlp_9hu/gapminder/displays/Gapminder/panels/lifeexp_time/albania_europe_al.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpuqlp_9hu/gapminder/displays/Gapminder/panels/lifeexp_time/algeria_africa_dz.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpuqlp_9hu/gapminder/displays/Gapminder/panels/lifeexp_time/angola_africa_ao.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpuqlp_9hu/gapminder/displays/Gapminder/panels/lifeexp_time/argentina_americas_ar.png
DEBUG:root:Saving image /v

### Add additional meta columns to the dataframe
To help with the analysis, we may want additional, meta columns. There is nothing here that is unique to Trelliscope, but rather this is a traditional data wrangling task such as adding summary statistics.

In [9]:
meta_df = df.groupby(["country", "continent", "iso_alpha2"]).agg(
    mean_lifeExp = ("lifeExp", "mean"),
    min_lifeExp = ("lifeExp", "min"),
    max_lifeExp = ("lifeExp", "max"),
    mean_gdp = ("gdpPercap", "mean"),
    first_year = ("year", "min"),
    latitude = ("latitude", "first"),
    longitude = ("longitude", "first")
)

meta_df = meta_df.reset_index()
meta_df["first_date"] = pd.to_datetime(meta_df["first_year"], format='%Y')
meta_df["wiki"] = meta_df["country"].apply(lambda x: f"https://en.wikipedia.org/wiki/{x}")
meta_df["country"] = meta_df["country"].astype("category")
meta_df["continent"] = meta_df["continent"].astype("category")


### Additional image columns
In addition to the meta columns, we can add additional image columns referring to URLs of online images or paths to local files. The following code downloads and extracts a zip file of flag images and also refers to online URLs to show both approaches.

In [10]:
# Download and extract flag images to a temporary directory
(zip_file, _) = urllib.request.urlretrieve("https://github.com/trelliscope/trelliscope/files/12265140/flags.zip")
local_flags_path = os.path.join(tempfile.mkdtemp(), "temp_flag_images")

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(local_flags_path)

# The flag column will hold references to the local files
meta_df["flag"] = meta_df["iso_alpha2"].apply(lambda x: os.path.join(local_flags_path, f"{x}.png"))

# The `flag_base_url` column will hold references to the remote URLs
flag_base_url = "https://raw.githubusercontent.com/hafen/countryflags/master/png/512/"
meta_df["flag_url"] = meta_df["iso_alpha2"].apply(lambda x: f"{flag_base_url}{x}.png")

print(meta_df[["country","flag", "flag_url"]].head())

       country                                               flag  \
0  Afghanistan  /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf...   
1      Albania  /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf...   
2      Algeria  /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf...   
3       Angola  /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf...   
4    Argentina  /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf...   

                                            flag_url  
0  https://raw.githubusercontent.com/hafen/countr...  
1  https://raw.githubusercontent.com/hafen/countr...  
2  https://raw.githubusercontent.com/hafen/countr...  
3  https://raw.githubusercontent.com/hafen/countr...  
4  https://raw.githubusercontent.com/hafen/countr...  


### Join the meta dataframe and the earlier panel dataframe.
Notice that if the index columns on each of these dataframes match they can be easily joined by Pandas.

In [11]:
meta_df = meta_df.set_index(["country", "continent", "iso_alpha2"])

joined_df = meta_df.join(panel_df)

### Create and view another Trelliscope, this time with the additional meta and image columns
When viewing this Trelliscope, notice that there are 3 different panels, one that is the plot of the Life Expectancy vs Time, and then two others for the flag images. The additional panels can be seen either by switching to a table view, or by selecting the "down arrow" in the top left corner of any of the panel images.

In [12]:
tr = (Trelliscope(joined_df, "Gapminder")
      .write_display()
      .view_trelliscope()
)

INFO:root:Using ['country', 'continent', 'iso_alpha2'] to uniquely identify each row of the data.
INFO:root:Saving to /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpqh152fxn/gapminder
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpqh152fxn/gapminder/displays/Gapminder/panels/lifeexp_time/afghanistan_asia_af.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpqh152fxn/gapminder/displays/Gapminder/panels/lifeexp_time/albania_europe_al.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpqh152fxn/gapminder/displays/Gapminder/panels/lifeexp_time/algeria_africa_dz.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpqh152fxn/gapminder/displays/Gapminder/panels/lifeexp_time/angola_africa_ao.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpqh152fxn/gapminder/displays/Gapminder/panels/lifeexp_time/argentina_americas_ar.png
DEBUG:root:Saving image /v

### Explicitly Set Trelliscope Parameters
In the previous examples, the default parameters were used for the Trelliscopes, but these parameters can be explicitly set if the default values are not sufficient.

In [13]:
tr = (Trelliscope(joined_df, name="Gapminder")
        .set_default_labels(["country", "continent"])
        .set_default_layout(3)
        .set_default_sort(["continent", "mean_lifeExp"], sort_directions=["asc", "desc"])
        .set_default_filters([NumberRangeFilterState("mean_lifeExp", 30, 60)])
        .write_display()
        .view_trelliscope()
)


INFO:root:Using ['country', 'continent', 'iso_alpha2'] to uniquely identify each row of the data.
INFO:root:Replacing entire existing sort state specification
INFO:root:Saving to /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpwjzcoyhr/gapminder
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpwjzcoyhr/gapminder/displays/Gapminder/panels/lifeexp_time/afghanistan_asia_af.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpwjzcoyhr/gapminder/displays/Gapminder/panels/lifeexp_time/albania_europe_al.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpwjzcoyhr/gapminder/displays/Gapminder/panels/lifeexp_time/algeria_africa_dz.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpwjzcoyhr/gapminder/displays/Gapminder/panels/lifeexp_time/angola_africa_ao.png
DEBUG:root:Saving image /var/folders/sd/q0zvkr053gn240n3g__01pn95pw3zf/T/tmpwjzcoyhr/gapminder/displays/Gapminder/panels/life